In [61]:
import pandas as pd
import string
import folium
import numpy as np

In [5]:
#importing data 
hoteltags=pd.read_pickle('./data/hoteltag.pkl')
geocode=pd.read_pickle('./data/geocode.pkl')

In [69]:
import geocoder

In [8]:
# merging geocode into dataset
hoteltags_geo= pd.merge(hoteltags,geocode,on='hotel_name',how='outer')

In [9]:
# merging geocode into dataset
hoteltags_geo= pd.merge(hoteltags,geocode,on='hotel_name',how='outer')

In [10]:
hoteltags_geo.head()

,hotel_name,tags,lat_x,lng_x,lat_y,lng_y,location,country,city
0,11 Cadogan Gardens,"[' Leisure trip ', ' Couple ', ' Superior Quee...",51.493616,-0.159235,51.493616,-0.159235,"[{'country_code': 'GB', 'city': 'Chelsea', 'co...",United Kingdom,Chelsea
1,1K Hotel,"[' Leisure trip ', ' Couple ', ' Superior M Do...",48.863932,2.365874,48.863932,2.365874,"[{'country_code': 'FR', 'city': 'Paris', 'coun...",France,Paris
2,25hours Hotel beim MuseumsQuartier,"[' Leisure trip ', ' Solo traveler ', ' Standa...",48.206474,16.354630,48.206474,16.354630,"[{'country_code': 'AT', 'city': 'Vienna', 'cou...",Austria,Vienna
3,41,"[' Leisure trip ', ' Couple ', ' Executive Kin...",51.498147,-0.143649,51.498147,-0.143649,"[{'country_code': 'GB', 'city': 'West End of L...",United Kingdom,West End of London
4,45 Park Lane Dorchester Collection,"[' Leisure trip ', ' Solo traveler ', ' Execut...",51.506371,-0.151536,51.506371,-0.151536,"[{'country_code': 'GB', 'city': 'West End of L...",United Kingdom,West End of London


In [11]:
#subsetting dataset
hoteltags_geo=hoteltags_geo[['hotel_name','tags','lat_x','lng_x','city']]

In [12]:
hoteltags_geo.head()

,hotel_name,tags,lat_x,lng_x,city
0,11 Cadogan Gardens,"[' Leisure trip ', ' Couple ', ' Superior Quee...",51.493616,-0.159235,Chelsea
1,1K Hotel,"[' Leisure trip ', ' Couple ', ' Superior M Do...",48.863932,2.365874,Paris
2,25hours Hotel beim MuseumsQuartier,"[' Leisure trip ', ' Solo traveler ', ' Standa...",48.206474,16.354630,Vienna
3,41,"[' Leisure trip ', ' Couple ', ' Executive Kin...",51.498147,-0.143649,West End of London
4,45 Park Lane Dorchester Collection,"[' Leisure trip ', ' Solo traveler ', ' Execut...",51.506371,-0.151536,West End of London


In [15]:
exclude = set(string.punctuation)
def clean(x):
    return set([''.join(ch for ch in i.lower() if ch not in exclude).strip() for i in x[2:][:-2].split(',')])

In [16]:
# applying to clean the tags and assigning them to a new column
hoteltags_geo['new_tags'] = hoteltags_geo['tags'].map(clean)


In [18]:
hoteltags_geo['new_tags'][0]

{'2 rooms',
 'business trip',
 'couple',
 'deluxe king room',
 'deluxe king suite',
 'family with older children',
 'family with young children',
 'group',
 'junior suite',
 'king suite',
 'leisure trip',
 'presidential suite',
 'single room',
 'solo traveler',
 'stayed 1 night',
 'stayed 2 nights',
 'stayed 3 nights',
 'stayed 4 nights',
 'stayed 5 nights',
 'stayed 6 nights',
 'stayed 7 nights',
 'stayed 8 nights',
 'stayed 9 nights',
 'submitted from a mobile device',
 'superior queen room'}

In [19]:

# global variable
tag_sum_list = []

def get_tag_sum_elems(tag_sum_string):
    global tag_sum_list # use the global variable
    # extend the global variable with this_list
    tag_sum_list.extend(tag_sum_string)
    return True

for i in hoteltags_geo['new_tags']:
    get_tag_sum_elems(i)

tag_sum_set = set(tag_sum_list)
print(tag_sum_set)
len(tag_sum_set)

{'suite with free bottle of champagne and spa fitness access', 'amsterdam street art queen guest room', 'suite without window', 'standard double room with view', 'standard family room', 'superior double room disability access', 'executive club double room', 'deluxe twin studio', 'standard double room with queen bed', 'superior double or twin room with balcony', 'minsters deluxe two bedroom suite', 'stayed 22 nights', 'special offer classic room', 'twin room with courtyard view', 'deluxe family room 2 adults 2 children', 'premium double twin room', 'double room audacieuse', 'double room annex', 'deluxe with two double beds', 'grand deluxe imperial room', 'la reflet junior suite', 'double room with shower', 'smart triple room', 'standard double room with two double beds', 'small single room', 'junior suite with spa access free wifi', 'mus e d orsay double room', 'garden view queen', 'junior suite with queen size bed', 'deluxe double room with courtyard view', 'superior double', 'luxury t

2400

In [20]:
# removing 'submitted from a mobile device' since this attribute has contributing factor to our investigation
hoteltags_geo['new_tags'].apply(lambda x: x.remove('submitted from a mobile device'))

0       None
1       None
2       None
3       None
4       None
        ... 
1469    None
1470    None
1471    None
1472    None
1473    None
Name: new_tags, Length: 1474, dtype: object

In [21]:
#function to output the list of hotel with the respective attribute
def get_special(s):
    spike_cols = [col for col in tag_sum_set if s in col]
    hotel_list = set()
    for i in range(0, len(hoteltags_geo.tags)):
        for j in range(0,len(spike_cols)):
            if spike_cols[j] in hoteltags_geo.new_tags[i]:
                hotel_list.add(hoteltags_geo[hoteltags_geo.index==i]['hotel_name'][i])
                
    return hoteltags_geo[hoteltags_geo.hotel_name.isin(list(hotel_list))][['hotel_name','lat_x','lng_x','city']]


In [23]:
#plotting function to map out the location
def get_map(df,imagepath):
    if df.shape[0]<15:
            map2 = folium.Map(location=[df.iloc[0].lat_x,df.iloc[0].lng_x], zoom_start=12)
    elif df.shape[0]>15:
        map2 = folium.Map(location=[df.iloc[0].lat_x,df.iloc[0].lng_x], zoom_start=2)

    #generate folium
    map2 = folium.Map(location=[df.iloc[0].lat_x,df.iloc[0].lng_x], zoom_start=12)
    folium.raster_layers.TileLayer('Open Street Map').add_to(map2)
    for i in range(0,len(df)):
        folium.Marker(
        location=[df.iloc[i].lat_x,df.iloc[i].lng_x],
            tooltip=df.iloc[i].hotel_name,
        icon=folium.Icon(icon_color='white')
    ).add_to(map2)
    map2.save(imagepath)
    return map2


In [24]:
grp_col=['single room','river view','private pool','breakfast','spa bath','twin room','double room'
       ,'superior room','king room','executive room','city view','sea view','stayed '
       ,'eiffel twin','eiffel tower view','suite','triple room','penthouse','standard room','wheelchair accessible',
      'family room ','deluxe room','apartment','terrace']
newarray=[]

for i in grp_col:
    newarray.extend([col for col in tag_sum_set if i in col])

In [25]:
grp_col=['single room','river view','private pool','breakfast','spa bath','twin room','double room'
       ,'superior room','king room','executive room','city view','sea view','stayed '
       ,'eiffel twin','eiffel tower view','suite','triple room','penthouse','standard room','wheelchair accessible',
      'family room ','deluxe room','guest room','apartment','terrace']


In [26]:
#function to refactor the attributes
def new_fn(a,b):
    efv= list(get_special(a).index)
    for i in efv:
        hoteltags_geo.iloc[i].new_tags.add(b)

In [30]:
hoteltags_geo.iloc[323].new_tags

{'2 rooms',
 '4 rooms',
 'business trip',
 'classic double room',
 'classic single room',
 'classic twin room',
 'comfort double room',
 'couple',
 'deluxe double room with eiffel tower view',
 'deluxe eiffel twin',
 'deluxe twin room with eiffel tower view',
 'family with older children',
 'family with young children',
 'group',
 'leisure trip',
 'solo traveler',
 'stayed 1 night',
 'stayed 2 nights',
 'stayed 3 nights',
 'stayed 4 nights',
 'stayed 5 nights',
 'stayed 6 nights',
 'suite',
 'suite with eiffel tower view',
 'superior double room',
 'travelers with friends'}

In [31]:
# to group/refactor hotel attributes to new attributes
new_fn("single room","single_room")
new_fn("river view","river_view")
new_fn("private pool","private_pool")
new_fn("breakfast","break_fast")
new_fn("spa bath","spa_bath")
new_fn("twin room","twin_room")
new_fn("double room","double_room")
new_fn("superior room","superior_room")
new_fn("king room","king_room")
new_fn("executive room","executive_room")
new_fn("city view","city_view")
new_fn("sea view","sea_view")
new_fn("eiffel tower view","eiffel_tower_view")
new_fn("suite",'_suite_')
new_fn("triple room",'triple_room')
new_fn("penthouse",'_penthouse_')
new_fn("standard room",'standard_room')
new_fn("wheelchair accessible",'wheelchair_accessible')
new_fn("family room ",'family_room')
new_fn("deluxe room",'deluxe_room')
new_fn("guest room",'guest_room')
new_fn("apartment",'_apartment_')
new_fn("terrace",'_terrace')


In [32]:
# to remove attributes which are not used after replacing with generic attribute
for i in range(0, len(hoteltags_geo.new_tags)):
    for j in range(0,len(newarray)):
        if newarray[j] in hoteltags_geo.new_tags[i]:
            hoteltags_geo.new_tags[i].remove(newarray[j])

In [33]:
# global variable
tag_sum_list = []

def get_tag_sum_elems(tag_sum_string):
    global tag_sum_list # use the global variable
    # extend the global variable with this_list
    tag_sum_list.extend(tag_sum_string)
    return True

for i in hoteltags_geo['new_tags']:
    get_tag_sum_elems(i)

tag_sum_set = set(tag_sum_list)
len(tag_sum_set)

514

In [34]:
for i in tag_sum_list:
    hoteltags_geo[i] = 0

C:\Users\WAQARC~1\AppData\Local\Temp/ipykernel_7048/2576507773.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  hoteltags_geo[i] = 0


In [35]:
hoteltags_geo.head()

,hotel_name,tags,lat_x,lng_x,city,new_tags,leisure trip,couple,business trip,solo traveler,...,twin grand premier room,twin premier room,twin grand premier with canal view,king superior plus room,king loft,luxury quadruple room,large room,art room xl with iconic view,art room with iconic view,penta plus room
0,11 Cadogan Gardens,"[' Leisure trip ', ' Couple ', ' Superior Quee...",51.493616,-0.159235,Chelsea,"{leisure trip, couple, business trip, solo tra...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1K Hotel,"[' Leisure trip ', ' Couple ', ' Superior M Do...",48.863932,2.365874,Paris,"{leisure trip, couple, business trip, solo tra...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,25hours Hotel beim MuseumsQuartier,"[' Leisure trip ', ' Solo traveler ', ' Standa...",48.206474,16.354630,Vienna,"{city_view, leisure trip, with a pet, couple, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41,"[' Leisure trip ', ' Couple ', ' Executive Kin...",51.498147,-0.143649,West End of London,"{leisure trip, couple, business trip, solo tra...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,45 Park Lane Dorchester Collection,"[' Leisure trip ', ' Solo traveler ', ' Execut...",51.506371,-0.151536,West End of London,"{business trip, leisure trip, family with youn...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
#iterating through the columns comparing if attribtues in is presence in new_tags, assign 1 
only_new_tags=pd.DataFrame(hoteltags_geo.new_tags.loc[:])

for index, row in only_new_tags.iterrows():
    cs_list = set(row['new_tags'])
    
    for j in cs_list:
        #set corresponding column to 1
        if j in tag_sum_set:
            hoteltags_geo.loc[index,j]=1

In [37]:
hoteltags_geo.head()

,hotel_name,tags,lat_x,lng_x,city,new_tags,leisure trip,couple,business trip,solo traveler,...,twin grand premier room,twin premier room,twin grand premier with canal view,king superior plus room,king loft,luxury quadruple room,large room,art room xl with iconic view,art room with iconic view,penta plus room
0,11 Cadogan Gardens,"[' Leisure trip ', ' Couple ', ' Superior Quee...",51.493616,-0.159235,Chelsea,"{leisure trip, couple, business trip, solo tra...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1K Hotel,"[' Leisure trip ', ' Couple ', ' Superior M Do...",48.863932,2.365874,Paris,"{leisure trip, couple, business trip, solo tra...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,25hours Hotel beim MuseumsQuartier,"[' Leisure trip ', ' Solo traveler ', ' Standa...",48.206474,16.354630,Vienna,"{city_view, leisure trip, with a pet, couple, ...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,41,"[' Leisure trip ', ' Couple ', ' Executive Kin...",51.498147,-0.143649,West End of London,"{leisure trip, couple, business trip, solo tra...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,45 Park Lane Dorchester Collection,"[' Leisure trip ', ' Solo traveler ', ' Execut...",51.506371,-0.151536,West End of London,"{business trip, leisure trip, family with youn...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [38]:
index_room = ['single_room','twin_room','double_room','superior_room'
        ,'king_room','executive_room','_suite_','triple_room',
        '_penthouse_','standard_room','family_room','deluxe_room','guest_room','_apartment_',
        '_terrace_']
room_count=[]

for i in index_room:
    room_count.append(get_special(i).shape[0])


In [42]:
import plotly.io as pio


In [41]:
!pip install plotly


In [47]:
df1 = pd.DataFrame({'No.of Hotel': room_count}, index=index_room)
ax = df1.iplot(kind='barh', yTitle='Room Types', linecolor='black', title='Number of hotels by Room Types')
ax


In [45]:
!pip install cufflinks


  Created wheel for cufflinks: filename=cufflinks-0.17.3-py3-none-any.whl size=68734 sha256=34f3b8a946d2edf025ca921d53c969d8ab8076ae311231b09f53c56c5560a56b
  Stored in directory: c:\users\waqar computer\appdata\local\pip\cache\wheels\29\b4\f8\2fd2206eeeba6ccad8167e4e8894b8c4ec27bf1342037fd136
Successfully built cufflinks


In [46]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [48]:
index_view = ['river_view','private_pool','break_fast','spa_bath','city_view','sea_view','eiffel_tower_view'
              ,'wheelchair_accessible']
view_count=[]

for i in index_view:
    view_count.append(get_special(i).shape[0])

In [49]:
df2 = pd.DataFrame({'No.of Hotel': view_count}, index=index_view)
ax2 = df2.iplot(kind='barh', yTitle='Characteristic',colors='blue', linecolor='black', title='Number of hotels with special characteristics')
ax2

In [50]:
hoteltags_geo.isnull().sum()

hotel_name                      0
tags                            0
lat_x                           0
lng_x                           0
city                            0
                               ..
luxury quadruple room           0
large room                      0
art room xl with iconic view    0
art room with iconic view       0
penta plus room                 0
Length: 520, dtype: int64

In [51]:
hoteltags_geo.new_tags.iloc[323]

{'2 rooms',
 '4 rooms',
 '_suite_',
 'business trip',
 'couple',
 'double_room',
 'eiffel_tower_view',
 'family with older children',
 'family with young children',
 'group',
 'leisure trip',
 'single_room',
 'solo traveler',
 'travelers with friends',
 'twin_room'}

In [52]:
hoteltags_geo['double_room'].value_counts()

1    1046
0     428
Name: double_room, dtype: int64

In [54]:
# to generate map and visualise
get_map(get_special('eiffel_tower_view'),'./images/eiffel.html')

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
hoteltags_geo

,hotel_name,tags,lat_x,lng_x,city,new_tags,leisure trip,couple,business trip,solo traveler,...,twin grand premier room,twin premier room,twin grand premier with canal view,king superior plus room,king loft,luxury quadruple room,large room,art room xl with iconic view,art room with iconic view,penta plus room
0,11 Cadogan Gardens,"[' Leisure trip ', ' Couple ', ' Superior Quee...",51.493616,-0.159235,Chelsea,"{leisure trip, couple, business trip, solo tra...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1K Hotel,"[' Leisure trip ', ' Couple ', ' Superior M Do...",48.863932,2.365874,Paris,"{leisure trip, couple, business trip, solo tra...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,25hours Hotel beim MuseumsQuartier,"[' Leisure trip ', ' Solo traveler ', ' Standa...",48.206474,16.354630,Vienna,"{city_view, leisure trip, with a pet, couple, ...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,41,"[' Leisure trip ', ' Couple ', ' Executive Kin...",51.498147,-0.143649,West End of London,"{leisure trip, couple, business trip, solo tra...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,45 Park Lane Dorchester Collection,"[' Leisure trip ', ' Solo traveler ', ' Execut...",51.506371,-0.151536,West End of London,"{business trip, leisure trip, family with youn...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,citizenM London Bankside,"[' Leisure trip ', ' Couple ', ' Double Room '...",51.505151,-0.100472,City of London,"{leisure trip, with a pet, couple, business tr...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1470,citizenM London Shoreditch,"[' Leisure trip ', ' Solo traveler ', ' Double...",51.524137,-0.078698,Barbican,"{leisure trip, with a pet, couple, business tr...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1471,citizenM Tower of London,"[' Leisure trip ', ' Solo traveler ', ' Double...",51.510237,-0.076443,City of London,"{leisure trip, with a pet, couple, business tr...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1472,every hotel Piccadilly,"[' Leisure trip ', ' Couple ', ' Standard Doub...",51.510146,-0.131506,London,"{leisure trip, couple, business trip, solo tra...",1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [57]:
hoteltags_geo.iloc[:, 6:]

,leisure trip,couple,business trip,solo traveler,_suite_,single_room,family with young children,2 rooms,king_room,family with older children,...,twin grand premier room,twin premier room,twin grand premier with canal view,king superior plus room,king loft,luxury quadruple room,large room,art room xl with iconic view,art room with iconic view,penta plus room
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,1,1,1,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1470,1,1,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1471,1,1,1,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1472,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [58]:
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
similarityDF=cosine_similarity(hoteltags_geo.iloc[:, 6:],hoteltags_geo.iloc[:, 6:])


In [62]:
np.save("./data/tagcosine.npy",similarityDF)

In [63]:
hoteltags_geo.to_pickle("./data/clean_hoteltag.pkl")

In [64]:
def new_recommendations_tags(name,city, cosine_similarities):
    
    recommended_hotels = []
    
    #get input city index
    city_index= list(hoteltags_geo[hoteltags_geo.city==city].index)
    
    # gettin the index of the hotel that matches the name
    idx = hoteltags_geo[(hoteltags_geo.hotel_name == name)].index[0]
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    # getting the indexes of  similar hotels
    top_10_indexes = list(score_series.index)
    
    # populating the list with the names of the matching hotels
    for i in range(len(top_10_indexes)):
        if top_10_indexes[i] not in city_index:
            pass
        else:
            recommended_hotels.append(hoteltags_geo[hoteltags_geo.index==top_10_indexes[i]]['hotel_name'].values[0])

    # populating a dictionary of size 10 containing hotel name and lat and longitude 
    h = hoteltags_geo[['hotel_name','lat_x','lng_x']].to_dict(orient='records')
    l = {k['hotel_name']: [k['lat_x'], k['lng_x']] for k in h}
    if {hotel: l[hotel] for hotel in recommended_hotels }=={}:
        print("There are no hotels of similar hotel")
    else:
        output= {hotel: l[hotel] for hotel in recommended_hotels[:10]}
        newoutput={i:output for i in range(1,len(output)+1)}
        return newoutput
    

In [65]:
def get_hotel_fn(mydict,city):
    loc2 = geocoder.osm(city)

    # map
    main_map = folium.Map(location=[loc2.lat, loc2.lng], zoom_start=13)
    folium.raster_layers.TileLayer('Open Street Map').add_to(main_map)

    # loop through dict
    for i in range (1,len(mydict)+1):
        folium.Marker(location=list(mydict[i].values())[i-1],tooltip=list(mydict[i].keys())[i-1]
                      ,popup=list(mydict[i].keys())[i-1],
                     icon=plugins.BeautifyIcon(number=i,
                                               icon='bus',
                                            border_color='blue',
                                            border_width=0.5,
                                            text_color='red',
                                            inner_icon_style='margin-top:0px;')).add_to(main_map)
     
    return main_map


In [71]:
from folium import plugins

In [72]:
# to populate and pin recommended list of hotels
get_hotel_fn(new_recommendations_tags('Hilton Diagonal Mar Barcelona','Vienna',similarityDF),'Vienna')